In [953]:
import os
import re
import jaro
import folium
import numpy as np
import pandas as pd
import geopandas as gp
from fuzzywuzzy import fuzz
from tqdm.notebook import tqdm

def listdir_nohidden(path):
    def ld_nh(path):
        for f in os.listdir(path):
            if not f.startswith('.'):
                yield f
    return list (ld_nh(path))

def show_county_data(county, data):
    d = data[[c for c in data.columns if 'geo' not in c]].copy()
    try: out = d[d.county==county].sort_values(d.columns[1])
    except: out = d[d.county==county].sort_values(d.columns[1])
    return out

### import data

- `shapes` = precincts geojson
- `part` = pre-2020 demographic participation 
- `res` = nov 2020 results

Using the native indexes for mapping, so don't reset or drop them!

In [799]:
# SHAPES DATA

shapes = gp.read_file('electoral_precincts/2018Precincts.shp')
shapes = shapes[list(shapes.columns[:4])+['geometry']]
# rows where loc_prec contains a value not the same as prec_shp
mismatch = np.where(((shapes['locality']+','+shapes['prec_shp']
                  ).str.lower()!=shapes['loc_prec'].str.lower()))
print('>', len(mismatch[0]), 'rows found with unique loc_prec values. Overwriting prec_shp...')
# replace odd prec_shp values (they are duplicated in prec_elect)
shapes.prec_elec = shapes.loc_prec.apply(lambda x: x.split(',')[1])
shapes.drop('loc_prec', axis=1, inplace=True)

# dna = Duplicate Named Area (areas with unique geometry that )
dna = shapes[shapes.duplicated(['locality', 'prec_shp'])].copy()
dup_idx = dna[dna.duplicated(['geometry'])].index
dna.drop(dup_idx, inplace=True) # drop straight-up duplicates
shapes.drop(dup_idx, inplace=True) # drop straight-up duplicates
print('>', len(dna), 'with unique geometry rows share names! Adding indexes to names...')
idxr = 1
for d_idx in dna.index: # remainig rows have unique gemoetry
    o_val = dna.loc[d_idx, 'prec_shp']
    n_val = f'{o_val} ({idxr}/{len(dna)})'
    shapes.loc[d_idx, 'prec_shp'] = n_val # assign back into shapes directly.
    idxr += 1
    print('  >', n_val)
    
# clean vals (force lowercase) and rename
for c in shapes.columns[:3]:
    shapes[c] = shapes[c].str.lower()
shapes.rename(columns={'locality':'county'}, inplace=True)

shapes[shapes.county=='appling'].sort_values('prec_shp').head(3)


# OLD RUNOFF PARTICPATION DATA

part = pd.read_csv('../recent_runoffs/2018_november_cleaned/all_precincts_participation.csv')
part = part[list(part.columns[:3])]
for c in part.columns: part[c] = part[c].str.lower()
part.rename(columns={
        'County':'county',
        'PRECINCT ID': 'prec_id',
        'PRECINCT DESCRIPTION': 'prec_desc'
    }, inplace=True)

part[part.county=='appling'].sort_values('prec_desc').head(3)


# 2018 NOVEMBER RESULTS DATA

res = pd.read_csv('../2020_november/all_precincts_joined/US Senate (Loeffler) - Special.csv')
res = res[list(res.columns[:2])]
res.rename(columns={k:k.lower() for k in res.columns}, inplace=True)
for c in res.columns: res[c] = res[c].str.lower()
    
res[res.county=='appling'].sort_values('precinct').head(3)

> 6 rows found with unique loc_prec values. Overwriting prec_shp...
> 2 with unique geometry rows share names! Adding indexes to names...
  > PATRIOTS PARK (1/2)
  > PATRIOTS PARK (2/2)


,county,precinct
1450,appling,1b
1451,appling,1c
1452,appling,2


## manual fixes - exact string replacements

In [800]:
# REPLACE EXACT STRINGS in SHAPES DATA

replace_strs = {
    'hoggard mill': 'hoggards mill',
    'south mill': 'south milledgeville',
    'north mill': 'north milledgeville',
    'bethlehem church - 211': 'bethlehem church',
    'chattahoochee acvitity center': 'activity center' ,
    'cjc': '#3 cjc',
}
for o, n in replace_strs.items():
    for c in shapes.columns[:3]:
        shapes[c] = shapes[c].str.replace(o, n)
        
# REPLACE EXACT STRINGS in PART DATA

replace_strs = {
    'austin \(dun\)': 'austin',
    'avondale \(avo\)': 'avondale',
    'lithonia \(lit\)': 'lithonia',
    'woodward \(bhavn\)': 'woodward',
    'fbc - flc': 'family life center',
}
for o, n in replace_strs.items():
    for c in part.columns[:3]:
        part[c] = part[c].str.replace(o, n)
        
# REPLACE EXACT STRINGS in RES DATA
replace_strs = {
    ' ':' '
}
for o, n in replace_strs.items():
    for c in res.columns[:3]:
        res[c] = res[c].str.replace(o, n)

## direct edits by iloc - warning

In [801]:
# RISKY — THESE MAY CHANGE!!
part.loc[133, 'prec_desc'] = 'fairground'
part.loc[358, 'prec_desc'] = 'eli whitney'
part.loc[291, 'prec_desc'] = 'wilmington island presbyterian'
part.loc[347, 'prec_desc'] = 'wilmington island united'

res.loc[1136, 'precinct_id'] =  'bramlett elementary'
res.loc[1137, 'precinct_id'] =  'westside middle'


## specific county cleaning

### strip numbers from barrow precincts

In [802]:
# ADD COLUMN FOR PARSED PRECINCT IDs EXACT STRINGS
county='barrow'
barrow_idx = show_county_data(county, res).index
barrow_ids = res.loc[barrow_idx, 'precinct'].apply(lambda x: ' '.join(re.findall('[A-Za-z]*', x)))
res.loc[barrow_idx, 'precinct'] = barrow_ids

### de-code rockdale precincts

In [803]:
rockdale_p_map = {'BA': 'Barkside',
 'BT': 'Bethel',
 'CO': 'Conyers',
 'FI': 'Fieldstone',
 'FS': 'Flat Shoals',
 'HC': 'Honey Creek',
 'HI': 'High Tower',
 'LA': 'The Lakes',
 'LO': 'Lorraine',
 'MA': 'Magnet',
 'MI': 'Milestead',
 'OT': 'Olde Town',
 'RO': 'Rockdale',
 'SM': 'Smyrna',
 'SP': 'St. Pius',
 'ST': 'Stanton'}

def convert_rockdale(ab):
    ab = ab.upper()
    if ab in rockdale_p_map.keys():
        return rockdale_p_map[ab].lower()
    else:
        return ab.lower()
    
rock_idx = part[part.county=='rockdale'].index

part.loc[rock_idx, 'prec_id'] = part.loc[rock_idx, 'prec_id'].apply(convert_rockdale)

### fix spalding leading zero numbers

In [804]:
spald = shapes[shapes.county=='spalding'].index

def convert_spald(n):
    if len(n)<2:
        return'0'+n
    else:
        return n

shapes.loc[spald, 'prec_shp'] = shapes.loc[spald, 'prec_shp'].apply(convert_spald)

---

# search

## iterate shape index objects, looking for data from `part` and `res`

In [806]:
# THIS LOOP searches for matches across datasets using multiple columns
# (precinct id, precinct, prec_elec, prec_id, precinct description)
# ideally, a perfect match is found. always search within a matching county.
# if no perfect match for a given search, try fuzzy or custom searching... 

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# to avoid importing numpy!

        #   l = left data  |  r = right data

def find_matches(l_df, r_df, min_score, p_groupby='county', testing=False):
    match_hist = {} # to store the results and stats about matches
    found = 0
    customs = ['county', '_idx', '_mthd', '_fuzz'] # to exclude from search terms
    ignore_vals = ['88888', '99999', 'nan']

    l_df.fillna('nan', inplace=True)
    r_df.fillna('nan', inplace=True)
    
    if not testing: iter_over = sorted(list(l_df[p_groupby].unique()))
    else: iter_over=testing 
        
    if 'prec_desc' in r_df.columns: # first round, so save init INDEX
        l_df.reset_index(inplace=True)
        l_df.rename(columns={'index':'SHAPE_idx'}, inplace=True)

    for p_item in tqdm(iter_over):    

        # isolate data in just item group for both dataframes...
        l_group = l_df[l_df[p_groupby]==p_item] # geojson l_df data
        r_group = r_df[r_df[p_groupby]==p_item] # r_df grouped by county

        # for row in L_DATA data... (within this P_GROUP (ie COUNTY DATA)):
        for l_idx in l_group.index: # iterate items objects in this group
            l_row = l_group.loc[l_idx] 

            # to store results and break a search when a match is found
            match_hist[l_idx] = 0

            # search for PERFECT match across ALL COMPARISON COLUMNS:
            # this assumes there are no duplicates because it breaks when a match is found

        # PERFECT match search
            # iterating SHAPE precinct name columns...
            for l_compare in l_row.index: 
                if match_hist[l_idx]: break # break if perfect match found

                if any(cu in l_compare for cu in customs):
                    continue # ignore found data column

                l_val = l_row[l_compare] # SHAPE VALUE NAME
                if str(l_val).lower() in ignore_vals: continue

                # for row in R_DATA data...
                for r_idx in r_group.index: 
                    if match_hist[l_idx]: break # if search done, break this inner
                    r_row = r_group.loc[r_idx]

                    # ...compare against both possible precinct name columns
                    for r_compare in r_row.index:
                        if any(cu in r_compare for cu in customs):
                            continue # ignore found data column   
                        r_val = r_row[r_compare]
                        if str(r_val).lower() in ignore_vals: continue
                        if l_val == r_val: # ***CHECK THE L==R VALUE EQUATION***
                            meth = f'{l_compare} == {r_compare}'
                            
                            match_hist[l_idx] = {
                                'match_idx': r_idx, 'method': meth }
                            
                            l_df.loc[l_idx, 'r_idx'] = r_idx
                            l_df.loc[l_idx, 'r_mthd'] = meth
                            
                            break 

            # end l_row search if perfect match found
            if match_hist[l_idx]: continue


        # FUZZY match search (if needed)
            # minimum score to beat from function input
            best_score = min_score-.01
            if p_item=='fulton':
                best_score = .94
        
            for l_compare in l_row.index:  # iterating shape columns
                if any(cu in l_compare for cu in customs):
                    continue  # ignore found data column

                # no internal break, search ALL possible scores
                #if '_found' in l_compare: continue # ignore found data column

                l_val = l_row[l_compare] # SHAPE VALUE NAME
                if str(l_val).lower() in ignore_vals: continue

                for r_idx in r_group.index: # iterate R_DF rows
                    
                    if r_idx in l_df['r_idx'].values: continue # already assigned!
                        
                    # DONT break
                    r_row = r_group.loc[r_idx]
                    # search ALL possible FUZZY MATCHES...
                    for r_compare in r_row.index[1:]:
                        
                        if any(cu in r_compare for cu in customs):
                            continue # ignore results column(s)
                            
                        if best_score==1: break  # not going to find anything better!

                        r_val = r_row[r_compare] # PART VALUE NAME
                        if str(r_val).lower() in ignore_vals: continue
                        
                        # jaro score
                        f_score = jaro.jaro_metric(str(l_val), str(r_val))

                        if f_score > best_score: # new best score
                            best_score = f_score
                            f_str = str(round(f_score, 2)).split('.')[1]
                            meth = f"(F.{f_str}) {l_compare} = {r_compare}"
                            match_hist[l_idx] = {
                                'match_idx': r_idx,
                                'method': meth }
                                                    
                            l_df.loc[l_idx, 'r_idx'] = r_idx
                            l_df.loc[l_idx, 'r_mthd'] = meth
                            l_df.loc[l_idx, 'r_fuzz'] = r_val
                            # dont break, search all. 
                            
    l_df.replace('nan', pd.np.nan, inplace=True)
    r_df.replace('nan', pd.np.nan, inplace=True)

    
    return l_df, r_df

## function to join all parsed counties

In [807]:
def join_data(l_df, r_df, dname):
    data = pd.DataFrame()

    counties = sorted(list(l_df.county.unique()))
    for county in counties:
        lc_data = show_county_data(county, l_df)
        rc_data = show_county_data(county, r_df)

        #lc_data.drop('L_county', axis=1, inplace=True)

        c_data = pd.merge(rc_data, lc_data, left_index=True, right_on='r_idx', how='outer')

        data = pd.concat([data, c_data])

    data['county'] = data['county_x'].fillna(data['county_y']).copy()
    if 'prec_id' in data.columns: sorter = 'prec_id'
    else: sorter = 'precinct'
        
    data = data.sort_values(sorter).sort_values('county')
    
    data.reset_index(drop=True, inplace=True)
    data.drop(columns=['county_x', 'county_y'], inplace=True)
    
    # get found pct of total possible finds
    no_abs = data[(data.prec_desc!='88888') & (data.prec_desc!='99999')]
    if 'precinct' in data.columns:
        no_abs = data[(data.prec_desc!='88888') & (data.prec_desc!='99999') & (data.precinct.notna())]
    not_found = no_abs.r_idx.isna().sum()
    fd_pct = round( 100*( (len(no_abs)-not_found)/len(no_abs) ) , 3)
    print(f"{len(no_abs)-not_found}/{len(no_abs)} precincts matched ({fd_pct}%)")
    
    data.rename(columns={'r_idx':dname+'_idx'}, inplace=True)
    
    return data

# parse & merge

#### find matches in PART data

In [808]:
# takes just under 2 minutes with i5
l_df, r_df = find_matches(shapes, part, .7)
data = join_data(l_df, r_df, 'PART')


2676/2703 precincts matched (99.001%)


#### find matches in RES data

In [817]:
# takes around 45 seconds
data.drop(['r_mthd', 'r_fuzz'], axis=1, inplace=True)
l_df, r_df = find_matches(data, res, .65)

# merge the merged
merged = join_data(l_df, r_df, 'RES')


2672/2672 precincts matched (100.0%)


In [1153]:
merged = join_data(l_df, r_df, 'RES')

2672/2672 precincts matched (100.0%)


# define name types

In [1154]:
name_cols = list(merged.columns[:7])
name_cols.remove('SHAPE_idx')

# detect roman numerals to make uppercase
thousand = 'M{0,3}'
hundred = '(C[MD]|D?C{0,3})'
ten = '(X[CL]|L?X{0,3})'
digit = '(I[VX]|V?I{0,3})'
regex_pattern = r"%s%s%s%s$" % (thousand, hundred, ten, digit)	

import re
roman_check = lambda x: bool(re.match(regex_pattern, x.upper()))

def prettify(string):
    x = string.split(' ')
    out = []
    for y in x:
        z = y.split('-')
        out.append('-'.join([a.upper() if roman_check(a) or a.lower()=='us)'
                             else a.capitalize() for a in z]))
    return ' '.join(out)

def clean_long(string, county):

    for rem in re.findall('[\d][a-z]', string):
        string = string.replace(rem, '')

    for rem in re.findall('[\d]-', string):
        string = string.replace(rem, '')

    if county in ['chatham', 'columbia']:
        return re.sub('[\d]', '', string).strip()
        
    else:
        return string.strip()

for m_idx in merged.index: # iterate all rows
    
    m_row = merged.loc[m_idx]
    
    name_vals = [str(m_row[i]).strip() for i in name_cols]
    
    if '88888' in name_vals:
        longest = 'Absentee (domestic)'
        shortest = '88888'
    elif '99999' in name_vals:
        longest = 'Absentee (outside US)'
        shortest = '99999'
    
    else:
        longest = ' '*4
        shortest = ' '*8
        for i, name in enumerate(name_vals):
            if name=='nan': continue
            if len(name) > len(longest):
                longest = clean_long(name, m_row['county']) # remove extra numbers from longest
            if len(name) < len(shortest):
                shortest = name 
        shortest = shortest.strip()
        longest = longest.strip()
        
    # remove shortest from longest if overlap
    if len(longest) < len(shortest) and m_row['county']!='fayette':
        longest = shortest
        
    elif shortest != longest: # if unique val for shortest...
        
        if len(shortest)<5 and shortest in longest:
            longest.replace(shortest, '')
        
        if shortest not in longest:
            # assign shortest 
            merged.loc[m_idx, 'precinct_id'] = shortest.upper()    
    
    
        # check for alt names 
        alt = ''
        for i, name in enumerate(name_vals):
            if name=='nan': continue
            if len(shortest) < len(name) < len(longest):
                alt = clean_long(name, m_row['county'])
        
        if alt and alt not in longest:
            merged.loc[m_idx, 'precinct_alt'] = prettify(alt)
            
    # assign longest
    merged.loc[m_idx, 'precinct_name'] = prettify(longest)
 

### drop bad guesses
#### (when there is a better guess)

In [1155]:
all_finds = list(merged.RES_idx)
edited = []

for find in all_finds:
    
    these_finds = merged[merged.RES_idx==find]
    if len(these_finds)>1:
        
        best_idx = list(these_finds.sort_values(
            'r_mthd', ascending=False).index)[0] # most confident find
        
        for f_idx in these_finds.index:
            if f_idx != best_idx:
                merged.loc[f_idx, 'RES_idx'] = pd.np.nan
                edited.append(f_idx)
                
# isolate important columns
merged = merged[['county', 'precinct_name', 'precinct_id', 'precinct_alt'] + [c for c in merged.columns if 'idx' in c]]

# final sweep over long names vs codes

In [1156]:
for ridx in merged[merged.precinct_id.isna()==True].index:
    
    row = merged.loc[ridx]
    if type(merged.loc[ridx, 'precinct_name']) == float:
        merged.loc[ridx, 'precinct_name'] = int(merged.loc[ridx, 'precinct_name'])
               
    p_long = row['precinct_name']
    pl_split = p_long.split()
    out = ''
    if len(pl_split)>1: # multiple broken words
        out = ''
        for pl in pl_split:
            out += pl[0].upper()

            has_num = len([r for r in re.findall('[\d]*', pl) if r!= ''])
            if has_num:
                out = pl
                break
                
    else: # no space chars
        nums = [r for r in re.findall('[\d]*', p_long) if r!= '']
        has_num = len(nums)
        
        if has_num:
            if len(p_long)<=8:
                out = p_long
            else:
                out = nums[0]
        else:
            continue
                
    merged.loc[ridx, 'precinct_id'] = out
    
    if out == merged.loc[ridx, 'precinct_name']:
        merged.loc[ridx, 'precinct_name'] = pd.np.nan

        

In [1157]:
def intify(x):
    try: return str(int(float(x)))
    except: return x
    
merged.precinct_name = merged.precinct_name.apply(intify)
merged.precinct_id = merged.precinct_id.apply(intify)

# save

In [1161]:
merged.county = merged.county.apply(prettify)

merged['precinct_id'] = merged['precinct_id'].fillna(merged['precinct_name'])
merged = merged[[
            'county',
            'precinct_id',
            'precinct_name',
            'SHAPE_idx',
            'PART_idx',
            'RES_idx']]
merged.to_csv('unified_precinct_index.csv', index=False)